In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.datasets import make_blobs

# 2D

In [ ]:
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
# import some data to play with
iris = datasets.load_iris()
# Take the first two features. We could avoid this by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target

to_take_indices = list(set(np.arange(len(y)))-set(np.where(y==2)[0]))
X = X[to_take_indices]
y = y[to_take_indices]

#...different dataset (you may comment me out):
#X, y = make_blobs(n_samples=80, centers=2, random_state=6)

In [ ]:
def plot_all(X, model, title):
    X0, X1 = X[:, 0], X[:, 1]
    xx, yy = make_meshgrid(X0, X1)
    fig, ax = plt.subplots(1,1)
    plot_contours(ax, model, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Sepal length')
    ax.set_ylabel('Sepal width')
    ax.set_title(title)
    plt.show()
    return ax.get_xlim(), ax.get_ylim()
    

def plot_all_2(X, model, title):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = model.decision_function(xy).reshape(XX.shape)

    # plot decision boundary and margins
    ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
    # plot support vectors
    if hasattr(model, "support_vectors_"):
        ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], s=100, linewidth=1, facecolors='none', edgecolors='k')
    ax.set_title(title)
    plt.show()

In [ ]:
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)
xlim, ylim = plot_all(X, model, title='LinearSVC (linear kernel)')
plot_all_2(X, model, title='LinearSVC (linear kernel)')

In [ ]:
model = svm.SVC(kernel='linear', C=1).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)
xlim, ylim = plot_all(X, model, title='LinearSVC (linear kernel)')
plot_all_2(X, model, title='LinearSVC (linear kernel)')

In [ ]:
def abline(slope, intercept, xlim=(0,3), ylim=(0,3)):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    x_vals = np.array(axes.get_xlim())
    y_vals = -intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--')
    

def find_seperatrix(model):
    W=model.coef_[0]
    I=model.intercept_
    a = -W[0]/W[1]
    b = I[0]/W[1]
    return a, b

slope, intercept = find_seperatrix(model)

print(slope, intercept)

abline(slope, intercept, xlim, ylim)

In [ ]:
x_vals = np.array(xlim)
y_vals = -intercept + slope * x_vals
points = [np.array(i) for i in zip(x_vals, y_vals)]

vector = points[1]-points[0]

print(points)
print(vector)

In [ ]:
def show_vectors(ax, vectors, vectors_with_start):
    """https://stackoverflow.com/a/42282532/5122790"""
    HEAD_SIZE = 0.2
    M = np.array(vectors)
    colors = ['r','g','b','k','gray']
    if vectors:
        for i,l in enumerate(range(0,M.shape[0])):
            ax.arrow(0,0,M[i,0],M[i,1],head_width=HEAD_SIZE,head_length=HEAD_SIZE,color = colors[i], length_includes_head=True)

    colors = ['c','m','y']
    if vectors_with_start:
        for i,(xs,ys,xf,yf) in enumerate([(*i[0],*i[1]) for i in vectors_with_start]):
            ax.arrow(xs,ys,xf,yf,head_width=HEAD_SIZE,head_length=HEAD_SIZE,color = colors[i], length_includes_head=True)

    return ax

In [ ]:
def ortho_proj(a,b):
    #https://en.wikipedia.org/wiki/Vector_projection
    a1_sc = np.dot(a,(b/np.linalg.norm(b)))
    b_hat = b/np.linalg.norm(b)
    a1 = a1_sc*b_hat
    a2 = a-a1
    return a2

In [ ]:
center_point = points[0]+0.5*vector

In [ ]:
print(ortho_proj(vector, center_point))

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
show_vectors(ax, [], [(points[0], vector), (center_point, ortho_proj(vector, center_point))])

ax.plot(*center_point,'ok') 
ax.set_aspect('equal', 'box')
plt.grid(b=True, which='major')

plt.xlim(xlim)
plt.ylim(ylim)
plt.show()

In [ ]:
print(model.coef_[0])
print(center_point+model.coef_[0])

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
show_vectors(ax, [], [(points[0], vector), (center_point, ortho_proj(vector, center_point)), (center_point, model.coef_[0])])

ax.plot(*center_point,'ok') 
ax.set_aspect('equal', 'box')
plt.grid(b=True, which='major')

plt.xlim(xlim)
plt.ylim(ylim)
plt.show()

# 3D

In [ ]:
#https://stackoverflow.com/a/51301399/5122790 !!!
import plotly.express as px
import plotly.graph_objects as go
from mpl_toolkits import mplot3d

In [ ]:
# import some data to play with
iris = datasets.load_iris()
# Take the first 3 features. We could avoid this by using a two-dim dataset
X = iris.data[:, :3]
y = iris.target

to_take_indices = list(set(np.arange(len(y)))-set(np.where(y==2)[0]))
X = X[to_take_indices]
y = y[to_take_indices]

In [ ]:
fig = go.Figure()
fig.add_trace(
        go.Scatter3d(
            mode='markers',
            x=X[:,0],
            y=X[:,1],
            z=X[:,2],
            marker=dict(            
                color=y,
                size=2,
                line=dict(
                    width=0
                )
            ),
        )
    )

In [ ]:
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], color=["r" if i==0 else "b" for i in y])
plt.show()

In [ ]:
def plot_3d_boundary(clf, MESHGRID_AMT=30):
    # The equation of the separating plane is given by all x so that np.dot(svc.coef_[0], x) + b = 0.
    # Solve for w3 (z)
    z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
    # https://stackoverflow.com/a/51301399/5122790 !!!

    lsx = np.linspace(min(X[:,0])-0.5, max(X[:,0])-0.5, MESHGRID_AMT)
    lsy = np.linspace(min(X[:,1])-0.5, max(X[:,1])-0.5, MESHGRID_AMT)
    xx, yy = np.meshgrid(lsx,lsy)

    fig = plt.figure()
    ax  = fig.add_subplot(111, projection='3d')
    ax.plot3D(X[y==0,0], X[y==0,1], X[y==0,2],'ob')
    ax.plot3D(X[y==1,0], X[y==1,1], X[y==1,2],'sr')
    ax.plot_surface(xx, yy, z(clf,xx,yy))
    ax.view_init(30, 60)
    plt.show()
    
plot_3d_boundary(model)

In [ ]:
model.coef_[0]

In [ ]:
MESHGRID_AMT = 30
lsx = np.linspace(min(X[:,0])-0.5, max(X[:,0])-0.5, MESHGRID_AMT)
lsy = np.linspace(min(X[:,1])-0.5, max(X[:,1])-0.5, MESHGRID_AMT)
xx, yy = np.meshgrid(lsx,lsy)
z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
fig = go.Figure(data=[go.Surface(x=xx, y=yy, z=z(model,xx,yy))])
fig.add_trace(
        go.Scatter3d(
            mode='markers',
            x=X[:,0],
            y=X[:,1],
            z=X[:,2],
            marker=dict(            
                color=y,
                size=2,
                line=dict(
                    width=0
                )
            ),
        )
    )

In [ ]:
#https://stackoverflow.com/questions/43164909/plotlypython-how-to-plot-arrows-in-3d
vector = go.Scatter3d(x = [0,10],
                      y = [0,10],
                      z = [0,10],
                      marker = dict(size = 1),
                      line = dict(width = 6)
                     )

go.Figure(data=[vector])

In [ ]:
MESHGRID_AMT = 30
lsx = np.linspace(min(X[:,0])-0.5, max(X[:,0])-0.5, MESHGRID_AMT)
lsy = np.linspace(min(X[:,1])-0.5, max(X[:,1])-0.5, MESHGRID_AMT)
xx, yy = np.meshgrid(lsx,lsy)
z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
fig = go.Figure(data=[go.Surface(x=xx, y=yy, z=z(model,xx,yy))])
fig.add_trace(
        go.Scatter3d(
            mode='markers',
            x=X[:,0],
            y=X[:,1],
            z=X[:,2],
            marker=dict(            
                color=y,
                size=2,
                line=dict(
                    width=0
                )
            ),
        )
    )
fig.add_trace(
    go.Scatter3d(x = [X.mean(axis=0)[0]-model.coef_[0][0], model.coef_[0][0]+X.mean(axis=0)[0]],
                 y = [X.mean(axis=0)[1]-model.coef_[0][1], model.coef_[0][1]+X.mean(axis=0)[1]],
                 z = [X.mean(axis=0)[2]-model.coef_[0][2], model.coef_[0][2]+X.mean(axis=0)[2]],
                 marker = dict(size = 1),
                 line = dict(width = 6)
                 )
    )

## Now try to map points onto that vector.... uff

In [ ]:
def ortho_rejection_affine(first, second, candidate):
    """https://en.wikipedia.org/wiki/Vector_projection"""
    b = second-first
    a = candidate
    a1_sc = np.dot(a,(b/np.linalg.norm(b)))
    b_hat = b/np.linalg.norm(b)
    a1 = a1_sc*b_hat
    a2 = a-a1
    return a2*-1

def ortho_projection_affine(first, second, candidate):
    """https://en.wikipedia.org/wiki/Vector_projection"""
    b = second-first
    a = candidate
    a1_sc = np.dot(a,(b/np.linalg.norm(b)))
    b_hat = b/np.linalg.norm(b)
    a1 = a1_sc*b_hat
    return a1


def ortho_rejection_affine_2(first, candidate):
    """https://en.wikipedia.org/wiki/Vector_projection"""
    b = first
    a = candidate
    a1_sc = np.dot(a,(b/np.linalg.norm(b)))
    b_hat = b/np.linalg.norm(b)
    a1 = a1_sc*b_hat
    a2 = a-a1
    return a2*-1

def ortho_projection_affine_2(first, candidate):
    """https://en.wikipedia.org/wiki/Vector_projection"""
    a, b = first, candidate 
    return np.dot(np.dot(a,b)/np.dot(b,b),b)

In [ ]:
MESHGRID_AMT = 30
lsx = np.linspace(min(X[:,0])-0.5, max(X[:,0])-0.5, MESHGRID_AMT)
lsy = np.linspace(min(X[:,1])-0.5, max(X[:,1])-0.5, MESHGRID_AMT)
xx, yy = np.meshgrid(lsx,lsy)
z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]

layout = go.Layout(
            scene=dict(camera=dict(eye=dict(x=1, y=1, z=1)), #the default values are 1.25, 1.25, 1.25
                       xaxis=dict(),
                       yaxis=dict(),
                       zaxis=dict(),
                       aspectmode="data", #this string can be 'data', 'cube', 'auto', 'manual'
                       #a custom aspectratio is defined as follows:
                       #aspectratio=dict(x=1, y=1, z=0.95)
                    )
         )

fig = go.Figure(data=[go.Surface(x=xx, y=yy, z=z(model,xx,yy))], layout=layout)

#https://community.plotly.com/t/creating-a-3d-scatterplot-with-equal-scale-along-all-axes/15108/7
fig.update_layout(
    autosize=True,
    width=1000,
    height=800,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=10,
        pad=4
    ),
    paper_bgcolor="White",
)


fig.add_trace(
        go.Scatter3d(
            mode='markers',
            x=X[:,0],
            y=X[:,1],
            z=X[:,2],
            marker=dict(            
                color=y,
                size=2,
                line=dict(
                    width=0
                )
            ),
        )
    )
fig.add_trace(
    go.Scatter3d(x = [X.mean(axis=0)[0]-model.coef_[0][0], model.coef_[0][0]+X.mean(axis=0)[0]],
                 y = [X.mean(axis=0)[1]-model.coef_[0][1], model.coef_[0][1]+X.mean(axis=0)[1]],
                 z = [X.mean(axis=0)[2]-model.coef_[0][2], model.coef_[0][2]+X.mean(axis=0)[2]],
                 marker = dict(size = 1),
                 line = dict(width = 6)
                 )
    )
fig.add_trace(
    go.Scatter3d(x = [-model.coef_[0][0]*5, model.coef_[0][0]*5],
                 y = [-model.coef_[0][1]*5, model.coef_[0][1]*5],
                 z = [-model.coef_[0][2]*5, model.coef_[0][2]*5],
                 marker = dict(size = 1),
                 line = dict(width = 6)
                 )
    )
fig.add_trace(
    go.Scatter3d(x = [0],
                 y = [0],
                 z = [0],
                 marker = dict(size = 3, color="black"),
                 )
    )

print("Projecting onto:", model.coef_[0])
for n,point in enumerate(X):
    rej = ortho_rejection_affine_2(point, model.coef_[0])
    proj = ortho_projection_affine_2(point, model.coef_[0])
    fig.add_trace(
    go.Scatter3d(x = [point[0], proj[0]],
                 y = [point[1], proj[1]],
                 z = [point[2], proj[2]],
                 marker = dict(size = 1),
                 line = dict(width = 6)
                 )
    )
    
    
fig